# Установка и импорт необходимых моуделй



In [49]:
!pip install catboost
!pip install pymorphy2

In [50]:
# при работе через Google Colab
# прописать путь до всех файлов py
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Учёба/8 семестр/диплом/Проект/VKR/Блокноты')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import AnalysisFunc
import BaseFuncs
import PredictionSplit
import PredictionResults

In [52]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [53]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor

from itertools import chain

# Загрузка данных прошлого года и нынешнего
Для прошлого года создаются датафреймы с очищенными выбросами по Q и Z

In [54]:
data = BaseFuncs.GetDataFrameListFromFileConverted("/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/Преобразованные/ПиОА 1 преобразованные.csv")
data = AnalysisFunc.MakeFloat(data, ["Пол"])
data_Q = AnalysisFunc.DeleteByQuantile(data, 0.15)
data_Z = AnalysisFunc.DeleteByZ(data)
data_dict = {"Original": data, "Квартиль": data_Q, "По Z": data_Z}
data.head(5)

Контрольная работа 1: max = 26.0
Контрольная работа 2: max = 11.55
Контрольная работа 3: max = 22.0
929
Сдали: 775, не сдали: 154
Контрольная работа 1: max = 26.0
Контрольная работа 2: max = 13.0
Контрольная работа 3: max = 20.6


,Пол,Не сдал(-а),Посещение до Контрольная работа 1,Баллы до Контрольная работа 1,Процент баллов до Контрольная работа 1,Контрольная работа 1,Посещение до Контрольная работа 2,Баллы до Контрольная работа 2,Процент баллов до Контрольная работа 2,Контрольная работа 2,Посещение до Контрольная работа 3,Баллы до Контрольная работа 3,Процент баллов до Контрольная работа 3,Контрольная работа 3
0,1.0,0,0.93,0.160,0.52,0.1200,1.0,0.060,0.26,0.1200,0.43,0.10,0.43,0.120
1,1.0,0,1.00,0.035,0.11,0.0738,1.0,0.012,0.05,0.0449,1.00,0.00,0.00,0.075
2,1.0,0,0.00,0.000,0.00,0.0000,0.0,0.000,0.00,0.0000,0.00,0.00,0.00,0.000
3,0.0,0,1.00,0.150,0.48,0.1170,1.0,0.060,0.26,0.1180,1.00,0.08,0.35,0.155
4,0.0,0,0.86,0.160,0.52,0.0448,0.5,0.060,0.26,0.1008,1.00,0.08,0.35,0.150


In [55]:
dataReal = BaseFuncs.GetDataFrameListFromFileConverted("/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/Преобразованные/ПиОА тест преобразованные.csv")
dataReal = AnalysisFunc.MakeFloat(dataReal, ["Пол"])
dataReal.head(5)

,Пол,Не сдал(-а),Посещение до Контрольная работа 1,Баллы до Контрольная работа 1,Процент баллов до Контрольная работа 1,Контрольная работа 1,Посещение до Контрольная работа 2,Баллы до Контрольная работа 2,Процент баллов до Контрольная работа 2,Контрольная работа 2,Посещение до Контрольная работа 3,Баллы до Контрольная работа 3,Процент баллов до Контрольная работа 3,Контрольная работа 3
0,1.0,1,0.3,0.048,0.30,0.010,0.2,0.025,0.12,0.0000,0.25,0.093,0.26,0.000
1,0.0,0,0.7,0.160,1.00,0.091,0.8,0.090,0.43,0.1304,0.75,0.100,0.28,0.110
2,1.0,1,1.0,0.000,0.00,0.030,0.8,0.000,0.00,0.0000,0.12,0.000,0.00,0.000
3,1.0,1,0.9,0.020,0.12,0.019,1.0,0.085,0.40,0.0170,1.00,0.020,0.06,0.057
4,1.0,1,0.5,0.020,0.12,0.038,0.2,0.000,0.00,0.0000,0.12,0.000,0.00,0.000


# Словари моделей прогнозирования

In [56]:
models = {  "k-ближайших соседей": KNeighborsClassifier(leaf_size=20, metric='minkowski', n_neighbors=12, p=2, weights='uniform'),
            "Рандомный лес": RandomForestClassifier(max_depth=15, max_features=1, min_samples_leaf=3, min_samples_split=3, n_estimators=100),
            "CatBoost": CatBoostClassifier(iterations=31, learning_rate=0.1, silent=True, allow_writing_files=False),
            "AdaBoostClassifier": AdaBoostClassifier(base_estimator=LogisticRegression(), learning_rate=2.0, n_estimators=48)}
models_regres = { "k-ближайших соседей": KNeighborsRegressor(leaf_size=34, metric='minkowski', n_neighbors= 15, p=1, weights='uniform'),
            "Рандомный лес": RandomForestRegressor(bootstrap=True, max_depth=3, max_features=None, min_samples_leaf=2, min_samples_split=3, n_estimators=14),
            "CatBoostRegressor": CatBoostRegressor(iterations=61, learning_rate=0.1, depth=2, silent=True, allow_writing_files=False),
            "AdaBoostRegressor": AdaBoostRegressor(learning_rate=0.1, loss='linear', n_estimators=8), #), \
            "Ridge": Ridge(),  \
            "LinearRegression": LinearRegression()}

# ИНФО

**Разделение датафреймов на X и Y** происходит через функции:

`PredictionSplit.SplitTestTrainPass(data, exam, not_for_prediction)` (Y - отметки о сдаче)

`PredictionSplit.SplitTestTrainTest(data, exam, not_for_prediction)` (Y - отметки о баллах контрольной)

где `not_for_prediction` - список параметров, которые не должны выходить в выборку X

**Прогнозирование** производится через функции:


---

**Получение словаря с ошибками/точностями**, где ключи - названия моделей из словаря `models`
`PredictionResults.PredictionModel(X, Y, models, test_size, accuracy_type)`

и

`PredictionResults.PredictionModelNew(X, Y, x_, y_, models, accuracy_type)`


---


**Получение словаря с матрицами ошибок**, где ключи - названия моделей из словаря `models`
`PredictionResults.PredictionModelMatrix(X, Y, models, test_size)`

и

`PredictionResults.PredictionModelMatrixNew(X, Y, x_, y_, models)`


---


`accuracy_type` берется из словаря:


```
accuracy_dict = {
    "recall": recall_score,
    "f1": f1_score,
    "r2": r2_score,
    "mae": mean_absolute_error
}
```



# Прогнозирование train_split
Данные прошлого года делятся на тренировочную и тестовую выборку (проверка для оригинальных данных, так и для очищенных)

**Сдача дисциплины**

In [57]:
for key, d in data_dict.items():
    print(f"Данные: {key}")
    print("Прогнозирование сдачи дисциплины")
    results = pd.DataFrame(index=list(models.keys()), columns=AnalysisFunc.exams)
    results_corr_more_01 = pd.DataFrame(index=list(models.keys()), columns=AnalysisFunc.exams)

    for ex in AnalysisFunc.exams:
        results[ex] = ""
        results_corr_more_01[ex] = ""

        X, Y = PredictionSplit.SplitTestTrainPass(d, ex, ["Оценка"])
        pred_res = PredictionResults.PredictionModel(X, Y, models, 0.3, "recall")
        for key, val in pred_res.items():
            results[ex][key] = val


        X, Y = PredictionSplit.SplitTestTrainPass(d, ex, ["Пол", "Оценка"])
        pred_res = PredictionResults.PredictionModel(X, Y, models, 0.3, "recall")
        for key, val in pred_res.items():
            results_corr_more_01[ex][key] = val

    display(results)
    print("Без учета пола")
    display(results_corr_more_01)

Данные: Original
Прогнозирование сдачи дисциплины


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,0.239,0.196,0.217
Рандомный лес,0.196,0.304,0.37
CatBoost,0.152,0.413,0.391
AdaBoostClassifier,0.63,0.696,0.739


Без учета пола


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,0.13,0.196,0.261
Рандомный лес,0.217,0.348,0.478
CatBoost,0.174,0.261,0.304
AdaBoostClassifier,0.609,0.587,0.717


Данные: Квартиль
Прогнозирование сдачи дисциплины


...

,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,0.149,0.234,0.277
Рандомный лес,0.362,0.362,0.277
CatBoost,0.17,0.277,0.362
AdaBoostClassifier,0.681,0.723,0.83


Данные: По Z
Прогнозирование сдачи дисциплины


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,0.239,0.065,0.152
Рандомный лес,0.37,0.261,0.261
CatBoost,0.239,0.283,0.348
AdaBoostClassifier,0.543,0.565,0.609


Без учета пола


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,0.239,0.109,0.174
Рандомный лес,0.304,0.348,0.304
CatBoost,0.283,0.37,0.391
AdaBoostClassifier,0.565,0.587,0.609


**Результаты контрольной**

In [ ]:
for key, d in data_dict.items():
    print(f"Данные: {key}")
    print("Прогнозирование контрольных без учета оценок")
    results = pd.DataFrame(index=list(models_regres.keys()), columns=AnalysisFunc.exams)
    results_corr_more_01 = pd.DataFrame(index=list(models_regres.keys()), columns=AnalysisFunc.exams)

    for ex in AnalysisFunc.exams:
        results[ex] = ""
        results_corr_more_01[ex] = ""

        X, Y = PredictionSplit.SplitTestTrainTest(d, ex, ["Оценка"])
        pred_res = PredictionResults.PredictionModel(X, Y, models_regres, 0.3, "mae")
        for key, val in pred_res.items():
            results[ex][key] = val*100

        X, Y = PredictionSplit.SplitTestTrainTest(d, ex, ["Пол", "Оценка"])
        pred_res = PredictionResults.PredictionModel(X, Y, models_regres, 0.3, "mae")
        for key, val in pred_res.items():
            results_corr_more_01[ex][key] = val*100
    display(results)
    print("Без учета пола")
    display(results_corr_more_01)

Данные: Original
Прогнозирование контрольных без учета оценок


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,3.5,3.5,3.2
Рандомный лес,3.4,3.1,2.9
CatBoostRegressor,3.4,3.1,2.8
AdaBoostRegressor,3.4,3.2,2.9
Ridge,3.5,3.2,2.9
LinearRegression,3.5,3.1,2.9


Без учета пола


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,3.4,3.3,3.0
Рандомный лес,3.4,3.1,2.8
CatBoostRegressor,3.4,3.1,2.8
AdaBoostRegressor,3.5,3.1,2.9
Ridge,3.5,3.3,2.9
LinearRegression,3.5,3.1,2.9


Данные: Квартиль
Прогнозирование контрольных без учета оценок


...

,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,3.3,3.4,3.3
Рандомный лес,3.2,3.0,3.0
CatBoostRegressor,3.2,3.0,2.9
AdaBoostRegressor,3.2,3.1,3.0
Ridge,3.1,3.3,3.1
LinearRegression,3.1,3.1,2.9


Данные: По Z
Прогнозирование контрольных без учета оценок


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,3.1,3.3,3.2
Рандомный лес,3.1,3.1,3.0
CatBoostRegressor,3.1,3.0,2.8
AdaBoostRegressor,3.2,3.1,2.9
Ridge,3.1,3.1,3.0
LinearRegression,3.1,3.0,2.9


Без учета пола


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,3.2,3.3,3.2
Рандомный лес,3.1,3.0,3.0
CatBoostRegressor,3.1,3.0,2.8
AdaBoostRegressor,3.1,3.0,3.0
Ridge,3.2,3.1,3.0
LinearRegression,3.1,3.0,2.9


# Прогнозирование данных текущего года
Данные прошлого года считаются обучающими, а данные этого года - тестовыми

**Сдача дисциплины**

In [ ]:
print("Сравнение на реальных данных")
for key, d in data_dict.items():
    print(f"Данные: {key}")
    print("Прогнозирование сдачи дисциплины")
    results = pd.DataFrame(index=list(models.keys()), columns=AnalysisFunc.exams)

    for ex in AnalysisFunc.exams:
        X, Y, = PredictionSplit.SplitTestTrainPass(d, ex, ["Пол", "Оценка"])
        x_, y_ = PredictionSplit.SplitTestTrainPass(dataReal, ex, ["Пол", "Оценка"])
        results[ex] = ""
        pred_res_matrix = PredictionResults.PredictionModelMatrixNew(X, Y, x_, y_, models)
        pred_res = PredictionResults.PredictionModelNew(X, Y, x_, y_, models, "recall")

        for key, val in pred_res.items():
            results[ex][key] = str(pred_res[key]) + "    " + str(pred_res_matrix[key])

    display(results)

    print("Без учета процента набранных баллов:")
    for ex in AnalysisFunc.exams:
        X, Y, = PredictionSplit.SplitTestTrainPass(d, ex, ["Пол", "Оценка"]+AnalysisFunc.before)
        x_, y_ = PredictionSplit.SplitTestTrainPass(dataReal, ex, ["Пол", "Оценка"]+AnalysisFunc.before)
        results[ex] = ""
        pred_res_matrix = PredictionResults.PredictionModelMatrixNew(X, Y, x_, y_, models)
        pred_res = PredictionResults.PredictionModelNew(X, Y, x_, y_, models, "recall")

        for key, val in pred_res.items():
            results[ex][key] = str(pred_res[key]) + "    " + str(pred_res_matrix[key])

    display(results)


Сравнение на реальных данных
Данные: Original
Прогнозирование сдачи дисциплины


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,0.135 [[683 4]\n [237 37]],0.139 [[685 2]\n [236 38]],0.128 [[685 2]\n [239 35]]
Рандомный лес,0.215 [[686 1]\n [214 60]],0.124 [[687 0]\n [238 36]],0.157 [[684 3]\n [228 46]]
CatBoost,0.139 [[686 1]\n [236 38]],0.146 [[684 3]\n [234 40]],0.234 [[683 4]\n [210 64]]
AdaBoostClassifier,0.5 [[678 9]\n [137 137]],0.526 [[680 7]\n [130 144]],0.595 [[677 10]\n [111 163]]


Без учета процента набранных баллов:


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,0.179 [[678 9]\n [225 49]],0.182 [[683 4]\n [224 50]],0.15 [[684 3]\n [233 41]]
Рандомный лес,0.318 [[681 6]\n [208 66]],0.168 [[685 2]\n [221 53]],0.193 [[685 2]\n [220 54]]
CatBoost,0.193 [[682 5]\n [221 53]],0.197 [[683 4]\n [220 54]],0.234 [[682 5]\n [210 64]]
AdaBoostClassifier,0.555 [[613 74]\n [122 152]],0.631 [[599 88]\n [101 173]],0.723 [[608 79]\n [ 76 198]]


Данные: Квартиль
Прогнозирование сдачи дисциплины


...

,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,0.179 [[678 9]\n [225 49]],0.186 [[683 4]\n [223 51]],0.146 [[684 3]\n [234 40]]
Рандомный лес,0.372 [[675 12]\n [209 65]],0.215 [[684 3]\n [218 56]],0.182 [[683 4]\n [214 60]]
CatBoost,0.19 [[681 6]\n [222 52]],0.274 [[683 4]\n [199 75]],0.274 [[682 5]\n [199 75]]
AdaBoostClassifier,0.631 [[590 97]\n [101 173]],0.668 [[579 108]\n [ 91 183]],0.766 [[574 113]\n [ 64 210]]


Данные: По Z
Прогнозирование сдачи дисциплины


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,0.15 [[685 2]\n [233 41]],0.135 [[684 3]\n [237 37]],0.135 [[684 3]\n [237 37]]
Рандомный лес,0.241 [[685 2]\n [211 63]],0.146 [[686 1]\n [234 40]],0.157 [[687 0]\n [234 40]]
CatBoost,0.186 [[685 2]\n [223 51]],0.19 [[684 3]\n [222 52]],0.23 [[681 6]\n [211 63]]
AdaBoostClassifier,0.547 [[677 10]\n [124 150]],0.62 [[673 14]\n [104 170]],0.672 [[672 15]\n [ 90 184]]


Без учета процента набранных баллов:


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,0.179 [[678 9]\n [225 49]],0.179 [[683 4]\n [225 49]],0.15 [[684 3]\n [233 41]]
Рандомный лес,0.296 [[677 10]\n [199 75]],0.201 [[685 2]\n [226 48]],0.197 [[685 2]\n [224 50]]
CatBoost,0.215 [[681 6]\n [215 59]],0.226 [[682 5]\n [212 62]],0.241 [[682 5]\n [208 66]]
AdaBoostClassifier,0.602 [[600 87]\n [109 165]],0.65 [[591 96]\n [ 96 178]],0.745 [[594 93]\n [ 70 204]]


**Результаты контрольной**

In [ ]:
print("Сравнение на реальных данных")

for key, d in data_dict.items():
    print("Прогнозирование контрольных без процентов")
    results = pd.DataFrame(index=list(models_regres.keys()), columns=AnalysisFunc.exams)

    for ex in AnalysisFunc.exams:
        results[ex] = ""
        X, Y, = PredictionSplit.SplitTestTrainTest(d, ex, [ "Оценка"]+AnalysisFunc.before)
        x_, y_ = PredictionSplit.SplitTestTrainTest(dataReal, ex, ["Оценка"]+AnalysisFunc.before)
        pred_res = PredictionResults.PredictionModelNew(X, Y, x_, y_, models_regres, "mae")
        for key, val in pred_res.items():
            results[ex][key] = val *100

    display(results)

Сравнение на реальных данных
Прогнозирование контрольных без процентов


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,3.5,3.3,3.6
Рандомный лес,3.5,3.1,3.6
CatBoostRegressor,3.5,3.0,3.5
AdaBoostRegressor,3.5,3.1,3.6
Ridge,3.5,3.2,3.4
LinearRegression,3.5,3.1,3.4


Прогнозирование контрольных без процентов


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,3.5,3.3,3.7
Рандомный лес,3.5,3.1,3.7
CatBoostRegressor,3.5,3.0,3.5
AdaBoostRegressor,3.5,3.2,3.7
Ridge,3.6,3.2,3.4
LinearRegression,3.5,3.0,3.4


Прогнозирование контрольных без процентов


,Контрольная работа 1,Контрольная работа 2,Контрольная работа 3
k-ближайших соседей,3.5,3.3,3.6
Рандомный лес,3.5,3.1,3.7
CatBoostRegressor,3.5,3.0,3.5
AdaBoostRegressor,3.5,3.2,3.7
Ridge,3.5,3.2,3.4
LinearRegression,3.5,3.0,3.4


**Результаты контрольной**

Сохранение прогнозов в XLSX-файл

In [ ]:
# прогнозирование контрольных и соханение результатов
for key_data, d in data_dict.items():
    print(f"Данные: {key_data}")
    for ex in AnalysisFunc.exams:
        X, Y, = PredictionSplit.SplitTestTrainTest(d, ex, [ "Оценка"]+AnalysisFunc.before)
        x_, y_ = PredictionSplit.SplitTestTrainTest(dataReal, ex, ["Оценка"]+AnalysisFunc.before)
        results = pd.DataFrame(columns=["Real "+ex]+list(models_regres.keys()))
        results["Real "+ex] = dataReal[ex].round(2)* 100
        for key, model in models_regres.items():
            model.fit(X, Y)
            data_prediction = model.predict(x_)
            data_prediction[data_prediction < 0] = 0
            results[key] = data_prediction.round(2)* 100
            mae = round(PredictionResults.accuracy_dict["mae"](y_.round(2), data_prediction.round(2)), 3)* 100
            print(f"{key}: спрогнозировано; mae = {mae}")

        BaseFuncs.SaveFileCSV(path = f"/content/drive/My Drive/Учёба/8 семестр/диплом/Проект/VKR/Учет контрольных/",
            filename = f"ПиОА {key_data} ({ex}) DIV100", results = results, csv = False)

        display(results)

Данные: Original
k-ближайших соседей: спрогнозировано; mae = 3.5000000000000004
Рандомный лес: спрогнозировано; mae = 3.5000000000000004
CatBoostRegressor: спрогнозировано; mae = 3.5000000000000004
AdaBoostRegressor: спрогнозировано; mae = 3.5000000000000004
Ridge: спрогнозировано; mae = 3.5999999999999996
LinearRegression: спрогнозировано; mae = 3.4000000000000004


,Real Контрольная работа 1,k-ближайших соседей,Рандомный лес,CatBoostRegressor,AdaBoostRegressor,Ridge,LinearRegression
0,1.0,3.0,2.0,3.0,2.0,4.0,4.0
1,9.0,11.0,11.0,11.0,10.0,10.0,10.0
2,3.0,6.0,5.0,5.0,5.0,7.0,6.0
3,2.0,5.0,5.0,5.0,5.0,7.0,6.0
4,4.0,4.0,3.0,3.0,3.0,5.0,4.0
...,...,...,...,...,...,...,...
956,5.0,9.0,8.0,9.0,8.0,10.0,9.0
957,10.0,7.0,6.0,7.0,6.0,10.0,9.0
958,10.0,14.0,12.0,12.0,12.0,11.0,11.0
959,8.0,11.0,11.0,11.0,10.0,10.0,10.0


k-ближайших соседей: спрогнозировано; mae = 3.3000000000000003
Рандомный лес: спрогнозировано; mae = 3.1
CatBoostRegressor: спрогнозировано; mae = 3.1
AdaBoostRegressor: спрогнозировано; mae = 3.1
Ridge: спрогнозировано; mae = 3.2
LinearRegression: спрогнозировано; mae = 3.1


,Real Контрольная работа 2,k-ближайших соседей,Рандомный лес,CatBoostRegressor,AdaBoostRegressor,Ridge,LinearRegression
0,0.0,1.0,3.0,3.0,3.0,3.0,3.0
1,13.0,14.0,12.0,11.0,12.0,11.0,11.0
2,0.0,8.0,5.0,4.0,4.0,7.0,6.0
3,2.0,7.0,6.0,7.0,4.0,7.0,7.0
4,0.0,2.0,4.0,3.0,4.0,5.0,4.0
...,...,...,...,...,...,...,...
956,0.0,8.0,8.0,9.0,8.0,10.0,9.0
957,10.0,11.0,12.0,12.0,12.0,12.0,13.0
958,9.0,13.0,12.0,12.0,12.0,12.0,11.0
959,12.0,11.0,12.0,10.0,12.0,9.0,9.0


k-ближайших соседей: спрогнозировано; mae = 3.5999999999999996
Рандомный лес: спрогнозировано; mae = 3.5999999999999996
CatBoostRegressor: спрогнозировано; mae = 3.5000000000000004
AdaBoostRegressor: спрогнозировано; mae = 3.6999999999999997
Ridge: спрогнозировано; mae = 3.4000000000000004
LinearRegression: спрогнозировано; mae = 3.4000000000000004


...

,Real Контрольная работа 1,k-ближайших соседей,Рандомный лес,CatBoostRegressor,AdaBoostRegressor,Ridge,LinearRegression
0,1.0,3.0,2.0,3.0,2.0,4.0,4.0
1,9.0,10.0,11.0,11.0,11.0,10.0,10.0
2,3.0,6.0,5.0,5.0,5.0,7.0,6.0
3,2.0,5.0,5.0,6.0,6.0,7.0,6.0
4,4.0,4.0,3.0,4.0,3.0,5.0,4.0
...,...,...,...,...,...,...,...
956,5.0,8.0,8.0,9.0,9.0,10.0,9.0
957,10.0,7.0,7.0,7.0,7.0,10.0,9.0
958,10.0,13.0,12.0,12.0,12.0,11.0,11.0
959,8.0,10.0,11.0,11.0,11.0,10.0,10.0


k-ближайших соседей: спрогнозировано; mae = 3.3000000000000003
Рандомный лес: спрогнозировано; mae = 3.1
CatBoostRegressor: спрогнозировано; mae = 3.0
AdaBoostRegressor: спрогнозировано; mae = 3.2
Ridge: спрогнозировано; mae = 3.2
LinearRegression: спрогнозировано; mae = 3.0


,Real Контрольная работа 2,k-ближайших соседей,Рандомный лес,CatBoostRegressor,AdaBoostRegressor,Ridge,LinearRegression
0,0.0,1.0,2.0,3.0,1.0,3.0,2.0
1,13.0,14.0,12.0,11.0,12.0,11.0,11.0
2,0.0,8.0,4.0,4.0,4.0,7.0,6.0
3,2.0,7.0,5.0,7.0,5.0,7.0,7.0
4,0.0,2.0,5.0,3.0,5.0,5.0,4.0
...,...,...,...,...,...,...,...
956,0.0,8.0,8.0,8.0,8.0,9.0,9.0
957,10.0,11.0,12.0,11.0,12.0,12.0,13.0
958,9.0,13.0,12.0,12.0,12.0,12.0,11.0
959,12.0,11.0,11.0,11.0,12.0,9.0,9.0


k-ближайших соседей: спрогнозировано; mae = 3.5999999999999996
Рандомный лес: спрогнозировано; mae = 3.5999999999999996
CatBoostRegressor: спрогнозировано; mae = 3.5000000000000004
AdaBoostRegressor: спрогнозировано; mae = 3.6999999999999997
Ridge: спрогнозировано; mae = 3.4000000000000004
LinearRegression: спрогнозировано; mae = 3.4000000000000004


,Real Контрольная работа 3,k-ближайших соседей,Рандомный лес,CatBoostRegressor,AdaBoostRegressor,Ridge,LinearRegression
0,0.0,1.0,2.0,1.0,1.0,1.0,1.0
1,11.0,13.0,14.0,14.0,13.0,12.0,13.0
2,0.0,2.0,1.0,0.0,1.0,0.0,0.0
3,6.0,5.0,3.0,4.0,3.0,6.0,5.0
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...
956,6.0,9.0,5.0,6.0,7.0,7.0,6.0
957,16.0,12.0,11.0,12.0,11.0,12.0,12.0
958,15.0,13.0,11.0,12.0,11.0,12.0,11.0
959,12.0,12.0,13.0,14.0,13.0,12.0,13.0
